# Docker Compose

## Docker Compose: la herramienta todo en uno

Todo lo que hemos hecho hasta ahora esta bien, pero es un poco tedioso. Para ello esta docker compose.

Si estas usando Windows esta instalada por defecto con docker desktop. Si estas usando Linux debes instalarla. Parece que ya la tengo. 

Docker Compose lo que nos permite es describir de forma declarativa la arquitectura de servicios que nuestra aplicacion necesita. Y se hace desde un pequeño archivo, para que docker corra casi todos los comandos que venimos ejecutando manualmente.

Vamos al proyecto en el **folder docker** donde hay un archivo llamado *docker-compose.yml*. Basicamente es una estructura declarativa.

![](https://i.imgur.com/rXZvpfz.png)

- services: indica los servicios que componen la aplicacion, en nuestro caso esta compuesta por dos servicios o microservicios: una que se llama **app** y otra que se llama **db** como en la seccion anterior.

- Donde se indica la imagen de mongo, le indico la version, pues ya tengo una imagen de dicha version.

Un servicio de compose es o viene siendo equivalente al concepto de container, donde **app** y **db** son dos contenedores. Aunque hay una sutil diferencia.

Porque un servicio, puede tener uno o mas contenedores de la misma imagen. Y tener por ejemplo mas contenedores de **app** para poder facilmente tener mas BW en nuestra aplicacion. 

- El statement *depends_on*, indicamos que dicho contenedor va a depender de el servicio *db*. Es decir, que **db** necesita ser creado antes que **app**

### Ejercicio.

Encontrar similitudes entre la figura anterior y los siguientes comandos usados anteriormente:

    docker run -d --name db mongo:5.0

    docker run -d --name app -p 3000:3000 --env MONGO_URL=mongodb://db:27017/test platziapp

### Aprendiendo a usar docker compose

#### Preparando el entorno

Empezemos por remover los contenedores previamente creados y evitar posibles conflictos:

![](https://i.imgur.com/39LyW8g.png)

    docker rm app db

Igualmente interesante, es lo descrito por el instructor, donde los procesos del ejercio anterior estan corriendo, y como al intentar crear un nuevo proceso usando los mismos puertos, pues va a generar un conflico

![](https://i.imgur.com/oMNxqh5.png)

#### Listo

Simplemente:

    docker compose up

Y empezara a correr y a llenar mi consola porque no lo corri en modo detach. Y empezara a mostrar los logs de ambas aplicaciones.

![](https://i.imgur.com/u9rCFJV.png)



    


- Crea los Containers, con un nombre propio, para evitar posibles conflictos con otros contenedores esten corriendo y tengan el mismo nombre. 

- Los containers empiezan por docker, no por docker, sino porque estoy en la carpeta docker 😏

- Fijate el orden crea los contenedores.

- A diferencia del instructor, no hay una evidencia haya creado una red

- y los atañe aparentemente a la misma red.

Si vamos al navegador en el *http://localhost:3000/* veremos esta corriendo 😎

Parece ser hay unas sutiles diferencias, de mis logs a los mostrados por el instructor. 

### Levantando el servicio en modo detach

    docker compose up -d

![](https://i.imgur.com/5QFMzye.png)

Y el resultado es el mismo:

![](https://i.imgur.com/ZkqFZLx.png)

#### Analizando 

![](https://i.imgur.com/HN9MRe8.png)

- app esta exponiendo el servicio en el puerto 3000 como era de esperarse y puedo accederlo desde mi maquina.

- pero db expone el puerto 27017 pero solo para la red interna.

¿Y cual es la red interna?


Inspeccionando cada una de las redes de docker encontre que:

    docker network inspect docker_default

![](https://i.imgur.com/Rz91spZ.png)

Los dos containers estan conectados en la red *docker default*



## Subcomandos de Docker Compose

👉 Importante aclarar que en el statement de la variable de entorno, en el compose file, **db** esta haciendo referencia al otro servicio, no necesito saber el nombre del contenedor(en perse) como fue fue el caso en la seccion de *docker network*

![](https://i.imgur.com/nS6sBTN.png)

### Ver los logs

Para ver lo de todos los servicios:

    docker compose logs 

Para ver solamente de un servicio:

    docker compose logs app

![](https://i.imgur.com/Y9nm16M.png)

Y para verlos en tiempo real, o a medida que se van generando:

    docker compose logs -f app

Y para el otro servicio obviamente:

    docker compose logs db

#### Corriendo un comando o proceso especifico en un contenedor

Muy facil, con exec, y en vez de colocar el nombre del contenedor, pues el nombre del servicio:

    docker compose exec app bash

Y efectivamente, vamos a poder ver nuestro codigo 

![](https://i.imgur.com/1WexGWK.png)






#### Borrando todo

    docker compose down 

E increiblemente, borra los contenedores tambien 😁

## Docker Compose como herramienta de desarrollo

### Remplazando image por build

supongamos realizamos unos cambios a nuestro codigo en el ultimo momento:

![](https://i.imgur.com/uCTsK2U.png)

Y para verlos reflejados tendriamos que buildiar la imagen, y hay si usar docker compose.

Sin embargo, *docker composer* nos permite un enfoque mas practico, modificando el *docker-compose.yml*:

![](https://i.imgur.com/xPzyNzl.png)

-  cambiamos la instruccion *image*, por *build*.
-  Como queremos darle un contexto a build que es el mismo directorio donde estamos trabajando, le colocamos el puntico.
- el servicio *db* se sigue asociando con una imagen.

Asegurarnos de estar en el folder del contexto de build, en mi caso *docker*, y:

    docker compose build

Tambien hubieramos podido usar, siendo especificos para que buildee un servicio especifico:

    docker compese build app

Y ahora si:

    docker compose up -d

No deberia tardar nada, y sin embargo, lo esta haciendo. Finalmente cuando lo hace verificamos cual fue la imagen que uso para buildiar app:


![](https://i.imgur.com/gbH08vt.png)

- anteriormente habia usado la imagen que teniamos en disco: *platziapp*, pero esta vez esta usanod la imagen que acabo de buildiar con *compose build*: **docker-app**

Pero tal vez, este enfoque no es el mas practico, porque queremos escribir codigo, y que esos cambios pasen directamente a la imagen o container que estamos trabajando, y no hacer un build cada vez.

### Volumes

A la altura del servico, agregamos una nueva instruccion, ojo con las indentaciones. Y hacemos un bind mount.

![](https://i.imgur.com/nxzfpxd.png)

- Que se monte lo que esta en mi ruta actual(.) en(:) usr/src

Y ahora si:

    docker compose up -d

Y todo deberia estar bien, pero no 😱

![](https://i.imgur.com/fQTCZvm.png)

#### Solucion

Algo se rompio, y no sabemos que fue:

- con *docker ps*, observamos solo esta corriendo el servico de mongo.
- investigando que paso con *docker compose logs app*

Dice que no puede encontrar un modulo, y esto paso en una clase anterior, y para solucionarlo hubo que montar solo *index.js*

Esto pasa porque se esta montando un directorio que no tiene instalado los modulos de node, y por tanto *node* no va poder arrancar porque le faltan esas dependencias. 

Con la siguiente instruccion le decimos que cuando hagas el *bind mount* ignore una ruta dentro del contenedor, que es donde esta los modulos cuando se hace el build. Y le estas diciendo que no montes nada encima de esa ruta.

![](https://i.imgur.com/ucaRzOE.png)

- fijate no lleva los dos puntos(:) ni nada

    docker compose up -d

Comprobamos que ambos servicioes esten corriendo, y vamos al navegador:

Y ahora si 😎

#### Verificando este tomando los cambios

hago un camio en mi codigo: *index.js* y miro si efectivamente esta tomando los cambios, y que crees que paso. NADA 😩😩

Si te acuerdas anteriormente, tuvimos que modificar el codigo para que *node*, y tambien en el *Dockerfile* para implementar que *nodedemon* monitoreara los cambios.

¿Habra una manera mas practica de hacerlo?

Agregamos una nueva instruccion:

![](https://i.imgur.com/CjMcbvT.png)

Si analizamos los logs de la aplicacion con:

    docker compose logs app

![](https://i.imgur.com/HlueM5C.png)

Ahora si esta funcionando como era su proposito 😏😏

## Compose en equipo: override

### Lectura:

Un **docker-compose.override.yml** es una herramienta valiosa en el ecosistema de Docker y Docker Compose. Te explicaré las ventajas de utilizarlo en tus proyectos:

1. Separación de Configuraciones: Permite separar la configuración base de Docker Compose (definida en docker-compose.yml) de las configuraciones específicas del entorno local o de desarrollo (definidas en docker-compose.override.yml). Esto facilita la gestión de diferentes configuraciones para diferentes entornos.

2. Personalización Local: Al utilizar un archivo docker-compose.override.yml, los miembros del equipo pueden personalizar su entorno local sin afectar la configuración base. Esto es útil para ajustar ciertos parámetros o añadir servicios adicionales necesarios para el desarrollo local.

3. Sobrescritura de Configuraciones: Cualquier configuración definida en docker-compose.override.yml sobrescribe las configuraciones correspondientes en el docker-compose.yml. Esto permite modificar fácilmente elementos como los comandos de inicio, los puertos expuestos o las variables de entorno.

4. Facilita la Colaboración: Cuando diferentes equipos o miembros del equipo trabajan en el mismo proyecto, el uso de un archivo docker-compose.override.yml facilita la colaboración sin interferir con las configuraciones de los demás.

5. Gestión de Diferentes Entornos: Es común que un proyecto tenga diferentes entornos (por ejemplo, desarrollo, pruebas, producción). Utilizando un docker-compose.override.yml, puedes mantener un conjunto base de configuraciones y luego crear archivos override específicos para cada entorno.

6. Mantiene el docker-compose.yml Limpio: Al mover configuraciones específicas del entorno local al archivo override, mantienes el docker-compose.yml más limpio y fácil de leer. Esto mejora la legibilidad y comprensión del archivo principal.

7. Facilita la Automatización: Cuando se integra con herramientas de automatización (como Jenkins, GitLab CI/CD u otras), puedes utilizar diferentes archivos docker-compose.override.yml para adaptar el comportamiento del despliegue a los diferentes entornos.

8. Pruebas y Desarrollo Iterativo: Facilita las pruebas y el desarrollo iterativo, ya que los cambios en la configuración local se pueden realizar sin afectar la configuración de otros entornos.

9. Documentación y Mantenimiento Simplificado: Al separar las configuraciones específicas del entorno, se mejora la documentación del proyecto y facilita el mantenimiento a largo plazo. Los desarrolladores pueden entender rápidamente las diferencias entre entornos.

10. Reduce Conflictos y Errores: Al permitir a los desarrolladores personalizar su entorno local, se reducen los posibles conflictos y errores causados por configuraciones incompatibles entre diferentes máquinas.

En resumen, el uso de un archivo docker-compose.override.yml proporciona flexibilidad y organización en la gestión de configuraciones específicas del entorno local en proyectos Docker, lo que facilita el desarrollo colaborativo y el despliegue en diferentes entornos.


### Como resolver el problema de la colaboracion con *Docker Compose*. 

1. Crear un nuevo archivo que se llame *docker-compose.override.yml* y asegurate tenga exactamente este nombre para no realizar configuraciones adicionales.

Si justo en este momento despues de crear este archivo vacio, realizamos:

    docker compose up

Nos dara un error:

![](https://i.imgur.com/NSOnSI7.png)

Esto es porque el archivo que acabamos de crear esta vacio, y Docker es muy estricto con su nomenclatura. Para el ejemplo, crearemos una nueva variable de entorno en *docker-compose.yml* que se llame *PLATZI_ENV*, y que tenga un valor de 2000.

    PLATZI_ENV: "2000"

Por otra parte en el *docker-compose.override.yml* haremos uso de la misma variable de entorno, pero con un valor diferente:

![](https://i.imgur.com/vH0Lw46.png)

y a continuacion para levantar todo:

    docker compose up -d 

![](https://i.imgur.com/CvZ0qg9.png)

Y posteriormente una terminal:

    docker compose exec app bash

Donde comprobaremos el valor de las variables de entorno con el comando **env**:

![](https://i.imgur.com/Pw8ptNm.png)

Observaremos el valor de la variable de entorno se sobreescribio con el arcivo de *override*. De tal forma que el *override* tiene prioridad sobre la base.


### Escalando un servicio a varias instancias

Suponga queremos tener dos instancias de *app*, con el comando, va a fallar:

    docker compose up -d --scale app=2

![](https://i.imgur.com/J9OkkTF.png)

Porque en mi *compose* le estoy diciendo que haga un bind del puerto 3000 al puerto 3000 de mi contenedor, y no puede haber mas de un contenedor escuchando en un puerto. 

Para que funcione hay que darle un rango, es decir:

![](https://i.imgur.com/zTa8cuQ.png)

Y volvamos a correr el comando. Si verificamos ahora en un navegador, el servicio estara disponible tanto en el puerto 3000 como 3001 de mi maquina host:

![](https://i.imgur.com/HmHhLxj.png)

Como ves, cada contenedor internamente esta exponiendo su puerto 3000 pero a un puerto diferente el mi maquina:

![](https://i.imgur.com/aiIRNnh.png)

Lo que acabamos de ver es un tipico ejemplo de escalamiento horizontal. 

### Caso Practico - pero cuidado ⛔

Supongamos que estás trabajando en un proyecto web con una base de datos y quieres tener configuraciones diferentes para tu entorno local y para el entorno de producción. 

Por ejemplo, en tu maquina se esta mapiando al puerto 3000, pero supongamos ya tenemos en uso ese puerto en mi maquina, y quiero cambiar el puerto, entonces no siempre funciona, pero lo intentaremos:

    version: "3.8"

    services:
    app:
        environment:
        PLATZI_ENV: "2023"
        ports:
        - "4000:3000"

Sin embargo, tal y como lo advirtio el profesor no funciono, porque puedo ver el servicio tanto en el puerto 3000 como 4000. Asi que no es una practica muy recomendable. ⛔

Mejor, define los puertos en un solo lugar, o en el compose, o el override.

### Image y BUild

Si Image y Build estan al mismo tiempo en un servicio, hara un *build*.

### Buena practica.

Agrega el *docker-compose.override.yml* al *gitignore*. Como este es un entorno de aprendizaje no lo hare.